In [1]:
import requests

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = requests.get(standings_url)

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(data.text)

In [6]:
standings_table = soup.select('table.stats_table')[0]

In [7]:
links = standings_table.find_all('a')

In [8]:
links = [l.get('href') for l in links]

In [9]:
links = [l for l in links if '/squads/' in l]

In [10]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [11]:
team_url = team_urls[0]

In [12]:
data = requests.get(team_url)

In [13]:
import pandas as pd 

matches = pd.read_html(data.text, match="Scores & Fixtures")

/var/folders/57/_ws0s1mx0xn9zc9gm8f422bw0000gn/T/ipykernel_18373/973186200.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")


In [14]:
soup = BeautifulSoup(data.text)

In [15]:
links = soup.find_all('a')

In [16]:
links = [l.get("href") for l in links]

In [17]:
links = [l for l in links if l and'all_comps/shooting/' in l]

In [18]:
data = requests.get(f"https://fbref.com{links[0]}")

In [19]:
shooting = pd.read_html(data.text, match="Shooting")[0]

/var/folders/57/_ws0s1mx0xn9zc9gm8f422bw0000gn/T/ipykernel_18373/2517714723.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


In [20]:
shooting.columns = shooting.columns.droplevel()

In [21]:
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [22]:
years = list(range(2022,2020, -1))

In [23]:
all_matches = []

In [24]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats" 

In [25]:
import time
import html5lib

for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get('href') for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select('a.prev')[0].get('href')
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")

        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]

        shooting.columns = shooting.columns.droplevel()

        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue

        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(10)
    



/var/folders/57/_ws0s1mx0xn9zc9gm8f422bw0000gn/T/ipykernel_18373/2696624384.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/57/_ws0s1mx0xn9zc9gm8f422bw0000gn/T/ipykernel_18373/2696624384.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
/var/folders/57/_ws0s1mx0xn9zc9gm8f422bw0000gn/T/ipykernel_18373/2696624384.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/57/_ws0s1mx0xn9zc9gm8f422bw0000gn/T/ipyke

In [26]:
match_df = pd.concat(all_matches)

In [27]:
match_df.columns = [c.lower() for c in match_df.columns]

In [28]:
match_df.to_csv("matches.csv")